In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--incognito --window-size=1920,1200 --headless")

browser = webdriver.Chrome(options=options)
browser.get("https://www.vaia.be/nl/opleidingen")
current_page = 1

In [23]:
value = "totaal aantal pagina"
pages = browser.find_elements(By.XPATH,"//*[contains(text(), '%s')]//ancestor::a[1]" % value)
pager = [page.get_attribute('innerHTML').split('</span>')[1] for page in pages]
total_pages = int(pager[0])
total_pages

81

In [25]:
courses = browser.find_elements(By.CLASS_NAME, "link--extended")
links = [elem.get_attribute('href') for elem in courses]
titles = [elem.get_attribute('innerHTML') for elem in courses]
links

['https://www.vaia.be/nl/opleidingen/dynamic-report-generation-with-r-markdown-and-quarto',
 'https://www.vaia.be/nl/opleidingen/symposium-on-ai-and-society',
 'https://www.vaia.be/nl/opleidingen/exploiting-sources-of-variation-in-your-data-the-anova-approach',
 'https://www.vaia.be/nl/opleidingen/inspiraition-netwerk',
 'https://www.vaia.be/nl/opleidingen/data-ai-in-marketing',
 'https://www.vaia.be/nl/opleidingen/artifici%C3%ABle-intelligentie-en-de-impact-op-onze-samenleving',
 'https://www.vaia.be/nl/opleidingen/lerend-netwerk-data-ai',
 'https://www.vaia.be/nl/opleidingen/boost-jouw-onderzoek-met-chatgpt']

In [26]:
course_dates = browser.find_elements(By.CLASS_NAME, "py-1")
course_date = [course_date.get_attribute('innerHTML').strip() for course_date in course_dates]
course_date

['22 december 2023',
 '22 december 2023',
 '9 januari 2024',
 '10 januari 2024',
 '11 januari 2024',
 '17 januari 2024',
 '18 januari 2024',
 '23 januari 2024']

In [27]:
def clean_meta_data(input):    
    return input[8:][:input[8:].find('<')].replace(",",";").replace(" - ","-").replace(" -","-")


course_meta_datas = browser.find_elements(By.CLASS_NAME, "redactor")
course_meta_data = [clean_meta_data(course_meta_data.get_attribute('innerHTML')) for course_meta_data in course_meta_datas]

browser.close()

course_meta_data.pop(0)
course_meta_data

['cursus-Gent-UGent',
 'symposium-Gent-UGent; VAIA; FAIR',
 'cursus-Gent-UGent',
 'webinar reeks-online-SBM',
 'navorming-Gent-HOGENT',
 'nascholing-online-CNO UAntwerpen; VAIA',
 'netwerkevent-Gent-Voka',
 '3 workshops-online-Vlaamse Hogescholenraad']

In [28]:
df = pd.DataFrame({'title':titles,
                   'url':links,
                   'date':course_date,
                   'course_type':[elem.split('-')[0] for elem in course_meta_data],
                   'location':[elem.split('-')[1] for elem in course_meta_data],
                   'organizer':[elem.split('-')[2] for elem in course_meta_data]})
                   
df['start_time'] = ""
df['language'] = ""
df['location_detail'] = ""
df['target_group'] = ""
df['subscription_limit'] = ""
df['constraints'] = ""
df['price'] = ""
df['details'] = ""
df['sub_title'] = ""
df['intro'] = ""



In [34]:
def get_details(url):
    browser = webdriver.Chrome(options=options)
    browser.get(url)        
    print(url)
    
    data = browser.find_elements(By.XPATH, "//div[contains(@class,'practical-info')]/div/div/div/div")
    course_meta_data = [course_meta_data.get_attribute('innerHTML') for course_meta_data in data]    
    
    data2 = browser.find_elements(By.XPATH, "//div[contains(@class,'practical-info')]/div/div/div[2]/div/div[1]/div/ul/li")
    course_meta_data2 = [elem.get_attribute('innerHTML') for elem in data2]    
    
    subscription, constraint, price, details, sub_title, intro =["","","","","",""]
    
    data3= browser.find_elements(By.XPATH, "//div[contains(@class,'practical-info')]/div/div/div[2]/div/div[1]/div/a")
    details = [elem.get_attribute('href') for elem in data3]
       
    for info in course_meta_data2:
        if "Inschrijvingen" in info:
            subscription = info
        elif "Voorwaarden" in info:
            constraint = info
        elif "Prijs" in info:
            price = info    
    
    browser.close()
    return course_meta_data[0], course_meta_data[1], course_meta_data[2], course_meta_data[3], subscription, constraint, price, details, sub_title, intro


def clean_data1(input):
    temp = input.strip()
    loc = temp.find('</svg>')
    temp = temp[loc+6:].strip()
    return temp

def clean_data2(input):
    temp = input.strip()
    temp = temp.replace("\n","")    
    return temp


for index, row in df.iterrows():
     
    start_time, location, language, target_group, subscription_limit, constraints, price, details, sub_title, intro = get_details(row['url'])

    df.loc[index, 'start_time'] = clean_data1(start_time)    
    df.loc[index, 'location_detail'] = clean_data1(location)
    df.loc[index, 'language'] = clean_data1(language)
    df.loc[index, 'target_group'] = clean_data1(target_group)
    df.loc[index, 'subscription_limit'] = clean_data2(subscription_limit)
    df.loc[index, 'constraints'] = clean_data2(constraints)
    df.loc[index, 'price'] = clean_data2(price)  
    df.loc[index, 'details'] = details
    df.loc[index, 'sub_title'] = clean_data1(sub_title)
    df.loc[index, 'intro'] = clean_data1(intro)
     
df         


# record 7 data inversed = manual clean-up of data
# date format
# remove doelgroep, voorwaarden, prijs, inschrijvingen
# check detail URL
# store body for NLP parsing
# loop for pages + wait between pages
# show some analytics on the data
# list unnique course types
# split price if range - calculate avarage price
# create feature for province
# detect people in body text

#DATA.to_csv(r'spain-wind.csv')

https://www.vaia.be/nl/opleidingen/dynamic-report-generation-with-r-markdown-and-quarto
https://www.vaia.be/nl/opleidingen/symposium-on-ai-and-society
https://www.vaia.be/nl/opleidingen/exploiting-sources-of-variation-in-your-data-the-anova-approach
https://www.vaia.be/nl/opleidingen/inspiraition-netwerk
https://www.vaia.be/nl/opleidingen/data-ai-in-marketing
https://www.vaia.be/nl/opleidingen/artifici%C3%ABle-intelligentie-en-de-impact-op-onze-samenleving
https://www.vaia.be/nl/opleidingen/lerend-netwerk-data-ai
https://www.vaia.be/nl/opleidingen/boost-jouw-onderzoek-met-chatgpt


,title,url,date,course_type,location,organizer,start_time,language,location_detail,target_group,subscription_limit,constraints,price,details
0,Dynamic Report Generation with R Markdown and ...,https://www.vaia.be/nl/opleidingen/dynamic-rep...,22 december 2023,cursus,Gent,UGent,22 dec 2023 09:00 - 16:00,Engels,"Krijgslaan 281, 9000 Gent",Doelgroep: This module targets anyone who want...,,Voorwaarden: Basic knowledge of R as provided ...,Prijs: €150 - € 330,TBI
1,Symposium on AI and Society,https://www.vaia.be/nl/opleidingen/symposium-o...,22 december 2023,symposium,Gent,UGent; VAIA; FAIR,22 dec 2023 09:15 - 14:30,Engels,Aula Academica of Ghent University,Doelgroep: people working in AI (including in ...,,Voorwaarden: a strong background in AI (techni...,Prijs: free,TBI
2,Exploiting Sources of Variation in your Data: ...,https://www.vaia.be/nl/opleidingen/exploiting-...,9 januari 2024,cursus,Gent,UGent,9 jan 2024 - 6 feb 2024,Engels,"Krijgslaan 281, 9000 Gent",Doelgroep: This course targets professionals a...,,Voorwaarden: Participants are expected to have...,Prijs: €495 - €1100,TBI
3,Inspir.ai.tion Netwerk,https://www.vaia.be/nl/opleidingen/inspiraitio...,10 januari 2024,webinar reeks,online,SBM,10 jan 2024 - 20 nov 2024,Nederlands,Online,Doelgroep: iedereen met enige ervaring in gene...,,Voorwaarden: u heeft een ChatGPT-workshop of i...,Prijs: € 295 – € 595,TBI
4,Data &amp; AI in marketing,https://www.vaia.be/nl/opleidingen/data-ai-in-...,11 januari 2024,navorming,Gent,HOGENT,11 jan 2024 - 14 mrt 2024,Nederlands,"Campus Mercator – Henleykaai 84, 9000 Gent","Doelgroep: marketeers, ondernemers, start-ups,...",,Voorwaarden: basiskennis marketing,Prijs: € 1300 – € 1500,TBI
5,Artificiële intelligentie en de impact op onze...,https://www.vaia.be/nl/opleidingen/artifici%C3...,17 januari 2024,nascholing,online,CNO UAntwerpen; VAIA,17 jan 2024 14:00 - 16:00,Nederlands,online (via Microsoft Teams),Doelgroep: breed publiek,,Voorwaarden: geen voorkennis nodig,Prijs: €44,TBI
6,Lerend Netwerk Data &amp; AI,https://www.vaia.be/nl/opleidingen/lerend-netw...,18 januari 2024,netwerkevent,Gent,Voka,18 jan 2024 - 30 jun 2024,Nederlands,"Voka Box - Lammerstraat 18, 9000 Gent",Doelgroep: strategisch verantwoordelijken voor...,,,Prijs: € 850,TBI
7,Boost jouw onderzoek met ChatGPT,https://www.vaia.be/nl/opleidingen/boost-jouw-...,23 januari 2024,3 workshops,online,Vlaamse Hogescholenraad,23 jan 2024 - 6 feb 2024,Online,9 uur uren,Nederlands,,Voorwaarden: Geen voorkennis nodig voor sessie...,Prijs: Gratis voor hogeschoolmedewerkers - And...,TBI
